In [1]:
import numpy as np
from itertools import product
import pandas as pd
import math

In [97]:
# Problem 4
S0 = 10
u = 1.05
d = 0.95
r = 0.01
N = 8
K = 11
q = (1+r-d)/(u-d)

w = np.array(list(product((0.95,1.05), repeat=8)))
SN = S0*np.cumprod(w, axis=1)
SN = np.insert(SN, 0, S0, axis=1)

B = np.array([11, 11.5, 12, 12.5, 13])
payoff = np.zeros((np.size(SN, 0), np.size(B)))
non_zero_count = np.zeros(np.size(B))

numH = (w == 1.05).sum(axis=1)
numT = 8-numH
risk_neutral_prob = np.zeros((np.size(SN, 0), np.size(B)))
risk_neutral_prob_B = np.zeros(np.size(B))
price = np.zeros(np.size(B))

for i in range(np.size(B)):
    for j in range(np.size(SN, 0)):
        if(np.amax(SN[j]) > B[i]):
            risk_neutral_prob[j,i] += q**numH[j]*(1-q)**numT[j]
            if(SN[j,8] > K):
                payoff[j,i] = SN[j,8]-K
                price[i] += risk_neutral_prob[j,i]*payoff[j,i]
                if(payoff[j,i] != 0):
                    non_zero_count[i] += 1
    risk_neutral_prob_B[i] = np.sum(risk_neutral_prob[:,i])
    
price = price/(1+r)**8
results = {'B Value':B, 'Non-zero Count':non_zero_count, 'Risk-Neutral Prob':risk_neutral_prob_B, 'No-Arbitrage Price':price}

df = pd.DataFrame(results)
dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
dfStyler

,B Value,Non-zero Count,Risk-Neutral Prob,No-Arbitrage Price
0,11,37,0.550771,0.465635
1,11.5,37,0.483494,0.465635
2,12,37,0.358525,0.465635
3,12.5,17,0.171072,0.314749
4,13,10,0.113841,0.261939


In [44]:
# Problem 6 Extra Credit
S0 = 1.05
u = 1.01
d = 0.99
r = 0
q = (1+r-d)/(u-d)

def sim_N_period_binom(N):
    w = np.array(list(product((d,u), repeat=N)))
    SN = S0*np.cumprod(w, axis=1)
    SN = np.insert(SN, 0, S0, axis=1)
    return SN

def hit_box_option_payoff(t, S, N):
    t1 = math.ceil(t[0])
    t2 = int(t[1])
    
    SN = sim_N_period_binom(N)
    SN_box = SN[:, t1:t2+1]
        
    box_check = np.where(np.logical_and(SN_box>=S[0], SN_box<=S[1]), 1, 0)
    payoff = np.any(box_check, axis=1)
    
    price = q**N*np.sum(payoff)/(1+r)**N
    return price

def miss_box_option_payoff(t, S, N):
    t1 = math.ceil(t[0])
    t2 = int(t[1])
    
    SN = sim_N_period_binom(N)
    SN_box = SN[:, t1:t2+1]
    
    box_check = np.where(np.logical_or(SN_box<=S[0], SN_box>=S[1]), 0, 1)
    payoff = 1-np.any(box_check, axis=1)
    
    price = q**N*np.sum(payoff)/(1+r)**N
    return price

print("The price of the Hit Box Option is: " + str(hit_box_option_payoff([1.9, 4.1], [1.03, 1.07], 4)))
print("The price of the Miss Box Option is: " + str(miss_box_option_payoff([1.9, 4.1], [1.045, 1.055], 4)))

The price of the Hit Box Option is: 0.75
The price of the Miss Box Option is: 0.375
